# 重新拉取空白行数据
1. 需修改: path和sheet_name
2. 结果保存到源目录，文件名为fixed_{path}

In [49]:
path = "data_村镇银行_0.xlsx"
sheet_name = "村镇银行"


import logging
import pandas as pd
import requests
import time
from concurrent.futures import ThreadPoolExecutor
from script import headers
from bs4 import BeautifulSoup

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='fix.log'
)

data = pd.read_excel(path, dtype={"邮政编码": str, "流水号": str,"变更前_流水号": str})
# driver = init_driver()
# max_retries = 5
# max_rate_limit_retries = 20  # 最多重试100次“操作过于频繁”
def fix(index, id):
    url = f"https://xkz.nfra.gov.cn/jr/NWkWkC/showLicenceInfo.do?id={id}"
    while True:
        times = 0
        try:
            response = requests.get(url, headers=headers)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'lxml').find('table', class_='trw-table-s1')
            if "您的操作过于频繁，请五分钟后再试" in soup.get_text():
                print(f"拉取频繁,id{id}, 第{times+1}次尝试连接, 正在重试") ####
                continue
            print(f"%(asctime)s请求成功,id{id}") ####
        except:
            print(f"请求失败,id{id}, 第{times+1}次尝试连接")  ####
            time.sleep(5)
            continue
        is_pre_section = False  # 标记是否处于变更前信息段落
        for tr in soup.find_all('tr'):
            # 检测变更前信息段落开始
            if tr.th and "变更前机构信息" in tr.th.get_text():
                is_pre_section = True
                continue
            # 处理数据行
            if 'a0' in tr.get('class', []):
                tds = tr.find_all('td')
                if len(tds) == 2:
                    key = tds[0].get_text(strip=True).replace('：', '')
                    value = tds[1].get_text(strip=True)
                    # 添加前缀规则
                    final_key = f"变更前_{key}" if is_pre_section else key
                    data.loc[index,final_key] = value
        response.close()
        break

with ThreadPoolExecutor(max_workers=10) as executor:
    for index, row in data.iterrows():
        if pd.isna(row["发证机关"]):
            executor.submit(fix, index, row["id"])

data.to_excel(f"fixed_{path}", index=False, sheet_name=sheet_name)

%(asctime)s请求成功,id182961
%(asctime)s请求成功,id21101
%(asctime)s请求成功,id9095
%(asctime)s请求成功,id9096
%(asctime)s请求成功,id243645
%(asctime)s请求成功,id106325
%(asctime)s请求成功,id153933
%(asctime)s请求成功,id59864
%(asctime)s请求成功,id235872
请求失败,id143252, 第1次尝试连接
%(asctime)s请求成功,id218521
请求失败,id194021, 第1次尝试连接
%(asctime)s请求成功,id152986
请求失败,id88935, 第1次尝试连接
请求失败,id222419, 第1次尝试连接
%(asctime)s请求成功,id191125
%(asctime)s请求成功,id147147
%(asctime)s请求成功,id220080
%(asctime)s请求成功,id126133
%(asctime)s请求成功,id195878
%(asctime)s请求成功,id182980
%(asctime)s请求成功,id103586
%(asctime)s请求成功,id215906
请求失败,id24946, 第1次尝试连接
%(asctime)s请求成功,id199350
%(asctime)s请求成功,id143252
%(asctime)s请求成功,id194021
%(asctime)s请求成功,id21120
%(asctime)s请求成功,id88935
%(asctime)s请求成功,id93177
%(asctime)s请求成功,id102994
%(asctime)s请求成功,id222419
%(asctime)s请求成功,id121893
%(asctime)s请求成功,id137761
%(asctime)s请求成功,id91521
%(asctime)s请求成功,id87599
%(asctime)s请求成功,id85558
%(asctime)s请求成功,id170952
%(asctime)s请求成功,id213395
%(asctime)s请求成功,id83756
%(asctime)s请求成功,id103001


# 测试

In [43]:
import pandas as pd
category= {"orgTypeName": "农村信用社"}
begin = 0
listDatas = pd.read_excel(f"idList{category["orgTypeName"]}_{begin//20000}.xlsx", dtype={"flowNo": str}).to_dict(orient="records")
print(listDatas[0:5])

[{'id': 194004, 'flowNo': '01140603', 'certCode': 'E0013U345090014', 'fullName': '北流市农村信用合作联社桂塘分社', 'address': nan, 'postCode': nan, 'setDate': '1995-12-21', 'printDate': '2025-02-21', 'useState': 3, 'organNo': nan, 'lastFlowNo': nan, 'shutOut': nan, 'exitDate': nan, 'lostReason': nan, 'simpleName': nan, 'deadLine': nan, 'makeReason': nan, 'longitude': nan, 'latitude': nan, 'englishName': nan, 'jrOrganPreproty': nan, 'jrOrganTypeNo': nan, 'jrOrgBranchTypeNo': nan, 'uscc': nan, 'scope': nan, 'lastScope': nan, 'fatherOrganNo': nan, 'fatherOrganName': nan, 'organName': nan, 'provinceName': nan, 'lastId': nan, 'lastCertCode': nan, 'lastfullname': nan, 'lastaddress': nan, 'colIndex': 1, 'date': '2025-02-21'}, {'id': 9071, 'flowNo': '01122027', 'certCode': 'E0130U213010029', 'fullName': '石家庄市栾城农村信用合作联社智汇城分社', 'address': nan, 'postCode': nan, 'setDate': '2013-11-19', 'printDate': '2025-02-21', 'useState': 3, 'organNo': nan, 'lastFlowNo': nan, 'shutOut': nan, 'exitDate': nan, 'lostReason': nan

# 拉取空白行数据的另一种方式
效率更低

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.edge.service import Service
# from selenium.webdriver.common.by import By
# from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# def init_driver():
#     service = Service()
#     driver = webdriver.Edge(service=service)
#     driver.set_page_load_timeout(30)  # 设置页面加载超时
#     return driver
#
# def wait_for_tbody(driver, timeout=10):
#     """等待 /html/body/div[2]/table/tbody 元素出现"""
#     try:
#         WebDriverWait(driver, timeout).until(
#             EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/table/tbody"))
#         )
#         return True
#     except TimeoutException:
#         print("等待 tbody 元素超时")
#         return False
#
# def check_rate_limit(driver):
#     """检查是否出现 '您的操作过于频繁，请五分钟后再试' 的提示"""
#     try:
#         element = driver.find_element(By.XPATH, "/html/body/div[2]/table/tbody/tr[2]/td")
#         if (element.get_attribute("colspan") == "2" and
#                 "您的操作过于频繁，请五分钟后再试" in element.text.strip()):
#             return True
#         return False
#     except NoSuchElementException:
#         return False
#
# def extract_data(driver):
#     """提取 /html/body/div[2]/table/tbody 下的信息"""
#     data = {}
#     try:
#         tbody = driver.find_element(By.XPATH, "/html/body/div[2]/table/tbody")
#         rows = tbody.find_elements(By.TAG_NAME, "tr")
#
#         for row in rows:
#             cells = row.find_elements(By.TAG_NAME, "td")
#             if len(cells) == 2:  # 确保是键值对形式的行
#                 field_name = cells[0].text.strip().rstrip("：")  # 去掉字段名后的冒号
#                 field_value = cells[1].text.strip()
#                 data[field_name] = field_value
#
#                 # 检查 "变更前机构信息" 部分的字段（id="td0" 到 id="td3"）
#                 row_id = row.get_attribute("id")
#                 if row_id in ["td0", "td1", "td2", "td3"]:
#                     data[f"变更前_{field_name}"] = field_value
#
#         return data
#     except NoSuchElementException:
#         print("未找到 tbody 元素，可能页面为空")
#         return {}
#     except Exception as e:
#         print(f"提取数据时出错: {e}")
#         return {}

#         retry_count = 0
#         rate_limit_retries = 0
#         while retry_count < max_retries:
#             try:
#                 driver.get(url)
#
#                 # 等待 tbody 出现
#                 if not wait_for_tbody(driver):
#                     print(f"ID {row['id']} 未加载到 tbody")
#                     continue
#
#                 # 检查是否被限制
#                 if check_rate_limit(driver):
#                     rate_limit_retries += 1
#                     if rate_limit_retries > max_rate_limit_retries:
#                         print(f"ID {row['id']} 超过最大频率限制重试次数，跳过")
#                         break
#                     print(f"ID {row['id']} 操作过于频繁，第 {rate_limit_retries} 次等待")
#                     time.sleep(5)  # 等待
#                     continue
#
#                 # 未被限制，提取数据
#                 dat = extract_data(driver)
#                 if dat:
#                     data.loc[index, "英文名称"] = dat.get('英文名称', '')
#                     data.loc[index, "机构住所"] = dat.get('机构住所', '')
#                     data.loc[index, "机构所在地"] = dat.get('机构所在地', '')
#                     data.loc[index, "邮政编码"] = dat.get('邮政编码', '')
#                     data.loc[index, "发证机关"] = dat.get('发证机关', '')
#                     data.loc[index, "业务范围"] = dat.get('业务范围', '')
#                     data.loc[index, "变更前_流水号"] = dat.get('变更前_流水号', '')
#                     data.loc[index, "变更前_机构编码"] = dat.get('变更前_机构编码', '')
#                     data.loc[index, "变更前_机构名称"] = dat.get('变更前_机构名称', '')
#                     data.loc[index, "变更前_机构地址"] = dat.get('变更前_机构地址', '')
#                     data.loc[index, "简称"] = dat.get('简称', '')
#                     print(f"ID {row['id']} 已爬取")
#                     break
#                 else:
#                     print(f"ID {row['id'],} 无数据")
#                     break
#
#             except TimeoutException:
#                 retry_count += 1
#                 print(f"ID {row['id']} 超时，重试 {retry_count}/{max_retries}")
#             except WebDriverException as e:
#                 retry_count += 1
#                 print(f"ID {row['id']} 浏览器错误: {e}，重试 {retry_count}/{max_retries}")
#             except Exception as e:
#                 print(f"ID {row['id']} 出错: {e}")
#                 break
#
#             if retry_count == max_retries:
#                 print(f"ID {row['id']} 超过最大重试次数，跳过")
#                 break
# driver.quit()
# data.to_excel(f"fixed_{path}", index=False)